<a href="https://colab.research.google.com/github/quagrain/Sign-Language-Research/blob/main/Image_Similarity_Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mean Squared Error

### Functions

In [ ]:
images = [
    '/content/clasp_1144x644.png',
    '/content/clasp_1152x650.png',
    '/content/sit_926x524.png',
    '/content/sit_930x522.png',
    '/content/thumbs_738X420.png',
    '/content/two_456x262.png',
    '/content/two_460x258.png',
    '/content/two_726x414.png',
    '/content/two_728x414.png',
    '/content/two_two_736x416.png',
    '/content/u1_1156x650.png',
    '/content/u2_1150x648.png',
    '/content/u3_930x522.png',
    '/content/u4_736x414.png',
    '/content/write_924x522.png',
    '/content/write_926x526.png',
    '/content/write_930x524.png'
]

In [ ]:
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
import cv2
import os

Mean Squared Error Function

In [ ]:
def mse(imageA, imageB):
    if imageA.shape != imageB.shape:
        imageB = cv2.resize(imageB, (imageA.shape[1], imageA.shape[0]))

    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)


    n = imageA.shape[0] * imageA.shape[1]  # width × height
    if len(imageA.shape) == 3:  # account for RGB by multiplyign channel by resolution
        n = n * imageA.shape[2]

    err /= float(n)
    return err

Given two images, show the images with their RGB and Greyscale MSEs

In [ ]:
def compare_images(imageA, imageB):
    imageA = cv2.imread(imageA)
    imageB = cv2.imread(imageB)

    mse_rgb = mse(imageA, imageB)
    mse_gray = mse(cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY), cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY))

    max_height = max(imageA.shape[0], imageB.shape[0])
    imageA = cv2.resize(imageA, (int(imageA.shape[1] * max_height / imageA.shape[0]), max_height))
    imageB = cv2.resize(imageB, (int(imageB.shape[1] * max_height / imageB.shape[0]), max_height))

    combined_image = np.concatenate((imageA, imageB), axis=1)
    cv2_imshow(combined_image)
    print(f"RGB MSE: {mse_rgb}")
    print(f"Grayscale MSE: {mse_gray}")
    print()

Given a list of image paths, show all the possible combinations of the images with their RGB and Greyscale MSEs

In [ ]:
def compare_all_images(image_list:list):
    for i in range(len(image_list)):
            for j in range(i + 1, len(image_list)):
                compare_images(image_list[i], image_list[j])

Store all the MSEs of every possible combination of the images in a DataFrame

In [ ]:
def get_dataframe(images):
    results = []
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            image_1_path = images[i]
            image_2_path = images[j]

            image_1_name = os.path.basename(image_1_path).split('.')[0]
            image_2_name = os.path.basename(image_2_path).split('.')[0]

            imageA = cv2.imread(image_1_path)
            imageB = cv2.imread(image_2_path)

            rgb_mse = mse(imageA, imageB)
            grayscale_mse = mse(cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY),
                                 cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY))

            results.append([image_1_name, image_2_name, rgb_mse, grayscale_mse])

    df = pd.DataFrame(results, columns=['image_1_name', 'image_2_name', 'rgb_mse', 'grayscale_mse'])
    return df

Show the results from the DataFrame

In [ ]:
def show_images_with_mse(row):
    image_1_path = f"/content/{row['image_1_name']}.png"
    image_2_path = f"/content/{row['image_2_name']}.png"

    image1 = cv2.imread(image_1_path)
    image2 = cv2.imread(image_2_path)

    max_height = max(image1.shape[0], image2.shape[0])
    image1 = cv2.resize(image1, (int(image1.shape[1] * max_height / image1.shape[0]), max_height))
    image2 = cv2.resize(image2, (int(image2.shape[1] * max_height / image2.shape[0]), max_height))

    combined_image = np.concatenate((image1, image2), axis=1)

    cv2_imshow(combined_image)

    print(f"Image 1: {row['image_1_name']}, Image2: {row['image_2_name']}")
    print(f"RGB MSE: {row['rgb_mse']:.2f}")
    print(f"Grayscale MSE: {row['grayscale_mse']:.2f}")

### Tests

In [ ]:
compare_images(images[4], images[7])

In [ ]:
df = get_dataframe(images)

In [ ]:
sorted = df.sort_values(by=['grayscale_mse'], ascending=[True]).head(40)

In [ ]:
sorted.head(10)

In [ ]:
#show_images_with_mse(sorted.iloc[0])

In [ ]:
for index, row in sorted.iterrows():
    show_images_with_mse(row)
    print("Index:", index)

## Structural Similarity Index Measure

### Functions

In [ ]:
from skimage import io
from skimage.metrics import structural_similarity as ssim

In [ ]:
imageA = io.imread(images[4])
imageB = io.imread(images[7])

In [ ]:
image1 = cv2.imread(images[4], cv2.IMREAD_UNCHANGED)
image2 = cv2.imread(images[7])

In [ ]:
similarity_index, _ = ssim(imageA, imageB, full=True)
print(f"Similarity Index: {similarity_index}")